<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Radiative-Transfer-Approximations-for-Two-Stream-Radiative-Transfer-in-Reflected-Light" data-toc-modified-id="Radiative-Transfer-Approximations-for-Two-Stream-Radiative-Transfer-in-Reflected-Light-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Radiative Transfer Approximations for Two-Stream Radiative Transfer in Reflected Light</a></span><ul class="toc-item"><li><span><a href="#Using-the-approx-key" data-toc-modified-id="Using-the-approx-key-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Using the <code>approx</code> key</a></span></li><li><span><a href="#How-to-see-what-radiative-transfer-scattering-options-exist?" data-toc-modified-id="How-to-see-what-radiative-transfer-scattering-options-exist?-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>How to see what radiative transfer scattering options exist?</a></span></li><li><span><a href="#Toon-Coefficients" data-toc-modified-id="Toon-Coefficients-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Toon Coefficients</a></span></li><li><span><a href="#Direct-Scattering-Approximation" data-toc-modified-id="Direct-Scattering-Approximation-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Direct Scattering Approximation</a></span></li><li><span><a href="#Multiple-Scattering-Approximations" data-toc-modified-id="Multiple-Scattering-Approximations-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Multiple Scattering Approximations</a></span></li><li><span><a href="#Raman-Scattering-Approximations" data-toc-modified-id="Raman-Scattering-Approximations-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Raman Scattering Approximations</a></span></li><li><span><a href="#The-Effect-of-Stellar-Spectrum" data-toc-modified-id="The-Effect-of-Stellar-Spectrum-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>The Effect of Stellar Spectrum</a></span></li></ul></li></ul></div>

# Radiative Transfer Approximations for Two-Stream Radiative Transfer in Reflected Light

Like any code, there are several approximations that go into computing intesity from various phase functions. In reflected light models, some of these approximations drastically change the output spectra. 

Here we go through each approximation to show what the effects are of each. Also, to show how to toggle between various inputs. 

In [1]:
import warnings
warnings.filterwarnings('ignore')
#picaso
from picaso import justdoit as jdi 
from picaso import justplotit as jpi 
from bokeh.plotting import show, figure
from bokeh.layouts import column
from bokeh.palettes import Colorblind8
from bokeh.io import output_notebook 
import astropy.units as u
output_notebook()

Loading BokehJS ...

## Using the `approx` key

In [2]:
opa = jdi.opannection(wave_range=[0.3,1])
cloud_free = jdi.inputs()

Notice that all the `approx` keys have predefined inputs. These are our recommendations for how to run the code. But, users should always be weary of these and test their sensitivity to your results.

## How to see what radiative transfer scattering options exist?

All of the approximations have to do with how scattering is handled. 

In [6]:
print('Options for Direct Scattring Phase: ', jdi.single_phase_options())

print('Options for Multiple Scattring Phase: ', jdi.multi_phase_options())

print('Options for Raman Scattering: ', jdi.raman_options())

print('Options for Toon89 Coefficients: ', jdi.toon_phase_coefficients())

Can also set functional form of forward/back scattering in approx['TTHG_params']
Options for Direct Scattring Phase:  ['cahoy', 'OTHG', 'TTHG', 'TTHG_ray']
Can also set delta_eddington=True/False in approx['delta_eddington']
Options for Multiple Scattring Phase:  ['N=2', 'N=1', 'isotropic']
Options for Raman Scattering:  ['oklopcic', 'pollack', 'none']
Options for Toon89 Coefficients:  ['quadrature', 'eddington']


### Set inputs normally

In [7]:
cloud_free.phase_angle(0) #phase in radians

cloud_free.gravity(gravity=25, gravity_unit=u.Unit('m/(s**2)' ))

#set star 
cloud_free.star(opa, 6000, 0.0122, 4.437)

#set atmosphere comp and temp 
cloud_free.atmosphere(filename=jdi.jupiter_pt(),delim_whitespace=True)

#make a copy to have a separate cloud input dict
from copy import deepcopy
cloudy=deepcopy(cloud_free)
cloudy.clouds( filename=jdi.jupiter_cld(),delim_whitespace=True)

## Toon Coefficients 

Though the default is Quadrature, Table 1 of [Toon et al. 1989]() gives two options for scattering phase functions for solar, reflected light.

In [9]:
#let's make two different figures for this
fig_cloudy = figure(x_range=[0.3,1],y_range=[0,1.2], width=500, height=300
                    ,x_axis_label='Wavelength [μm]', y_axis_label='Geometric Albedo',)
fig_no_cloud = figure(x_range=[0.3,1],y_range=[0,1.2], width=500, height=300
                      ,x_axis_label='Wavelength [μm]', y_axis_label='Geometric Albedo',)

#define our options
options = jdi.toon_phase_coefficients()
colors = Colorblind8[0:len(options)]

#loop through all approximations 
for approx, c in zip(options, colors):
    #set approximations
    cloud_free.approx(toon_coefficients = approx)    
    cloudy.approx(toon_coefficients = approx)  
    df = cloud_free.spectrum(opa)
    wno_nc, alb_nc = jdi.mean_regrid(df['wavenumber'] , df['albedo'],R=150)
    df = cloudy.spectrum(opa)
    wno_c, alb_c = jdi.mean_regrid(df['wavenumber'] , df['albedo'],R=150)
    fig_no_cloud.line(1e4/wno_nc, alb_nc, legend_label=approx, color=c, line_width=3)
    fig_cloudy.line(1e4/wno_c, alb_c,  color=c, line_width=3)
jpi.plot_format(fig_cloudy)
jpi.plot_format(fig_no_cloud)
show(column(fig_no_cloud,fig_cloudy ))

## Direct Scattering Approximation

The [derivation documentation](https://natashabatalha.github.io/picaso_dev#slide02) has a full description of these direct scattering approximations. Briefly I'll describe them here: 

At the center of each is the [One Term Henyey-Greenstein Phase Function (OTHG)](http://adsabs.harvard.edu/abs/1941ApJ....93...70H) and the [Two Term HG Phase Function (TTHG)](http://adsabs.harvard.edu/abs/1965ApJ...142.1563I). 

We also know that planet atmospheres have high degrees of Rayleigh scattering. [Cahoy+2010](http://adsabs.harvard.edu/abs/2010ApJ...724..189C) developed a methodology for incorporating Rayleigh into the direct scattering component. 

A more rrobust way of dealing with Rayleigh is to directly fold it's phase function into the TTHG phase function (TTHG_Ray). 

We'll run each case with and without a cloud so you can see what happens in both regimes

In [10]:
#let's make two different figures for this
fig_cloudy = figure(x_range=[0.3,1],y_range=[0,1.2], width=500, height=300
                    ,x_axis_label='Wavelength [μm]', y_axis_label='Geometric Albedo',)
fig_no_cloud = figure(x_range=[0.3,1],y_range=[0,1.2], width=500, height=300
                      ,x_axis_label='Wavelength [μm]', y_axis_label='Geometric Albedo',)

#define our options
options = jdi.single_phase_options()
colors = Colorblind8[0:len(options)]

#loop through all approximations 
for approx, c in zip(options, colors):
    #set approximations
    cloud_free.approx(single_phase = approx)    
    cloudy.approx(single_phase = approx)  
    df = cloud_free.spectrum(opa)
    wno_nc, alb_nc = jdi.mean_regrid(df['wavenumber'] , df['albedo'],R=150)
    df = cloudy.spectrum(opa)
    wno_c, alb_c = jdi.mean_regrid(df['wavenumber'] , df['albedo'],R=150)
    fig_no_cloud.line(1e4/wno_nc, alb_nc, legend_label=approx, color=c, line_width=3)
    fig_cloudy.line(1e4/wno_c, alb_c,  color=c, line_width=3)
jpi.plot_format(fig_cloudy)
jpi.plot_format(fig_no_cloud)
show(column(fig_no_cloud,fig_cloudy ))

Can also set functional form of forward/back scattering in approx['TTHG_params']


## Multiple Scattering Approximations

Again, [derivation documentation](https://natashabatalha.github.io/picaso_dev#slide03) has a full description of these multiple scattering approximations. 

To complete the multiple scattering integration over all _diffuse angles_, we have to use some mathematical tricks. [Legendre Polynomials](http://mathworld.wolfram.com/LegendrePolynomial.html) are often used to complete this integration to varying degrees. For Solar System/Exoplanet papers, we often stop the expansion at either `N=1` or `N=2`. Our standard will be to run with `N=2`, but below we show how to run each.

In [11]:
fig = figure(x_range=[0.3,1],y_range=[0,1.2], width=500, height=300,
             x_axis_label='Wavelength [μm]', y_axis_label='Geometric Albedo',)

options = jdi.multi_phase_options()
colors = Colorblind8[0:len(options)*2]

for approx, c1,c2 in zip(options, colors[0:2], colors[2:]):
    cloud_free.approx(multi_phase= approx)
    cloudy.approx(multi_phase = approx)
    df = cloud_free.spectrum(opa)
    wno_nc, alb_nc =jdi.mean_regrid( df['wavenumber'] , df['albedo'],R=150)
    df = cloudy.spectrum(opa)
    wno_c, alb_c =jdi.mean_regrid( df['wavenumber'] , df['albedo'],R=150)
    fig.line(1e4/wno_nc, alb_nc, color=c1, line_width=3)
    fig.line(1e4/wno_c, alb_c,  color=c2, line_width=3)
jpi.plot_format(fig)
show(fig)

Can also set delta_eddington=True/False in approx['delta_eddington']


## Raman Scattering Approximations

We all know the importance of Rayleigh scattering in planetary atmospheres. Raman scattering also has important implications for our spectra (these features have been observed in Solar System planets). In particular, at short wavelengths, Raman scattering imprints molecular features from the star on the planetary spectrum. 

The most complete analysis of all Raman approximations is in [Sromosvky+2005](http://adsabs.harvard.edu/abs/2005Icar..173..254S). From these, we use the _Pollack Approximation_ that was used in [Cahoy+2010](http://adsabs.harvard.edu/abs/2010ApJ...724..189C) and others. 

We include the original Pollack methodology, but also include a modified version with [Oklopcic et al 2018](http://iopscience.iop.org/article/10.3847/0004-637X/832/1/30/meta) cross sections and updated methodology to include effects of stellar spectrum.  

In [12]:
fig_cloudy = figure(x_range=[0.3,1],y_range=[0,1.2], width=500, height=300
                    ,x_axis_label='Wavelength [μm]', y_axis_label='Geometric Albedo',)
fig_no_cloud = figure(x_range=[0.3,1],y_range=[0,1.2], width=500, height=300
                      ,x_axis_label='Wavelength [μm]', y_axis_label='Geometric Albedo',)

options = jdi.raman_options()
colors = Colorblind8[0:len(options)]

for approx, c in zip(options, colors):
    cloud_free.approx(raman = approx)
    cloud_free.star(opa, 6000, 0.0122, 4.437)
    cloudy.approx(raman = approx)
    cloudy.star(opa, 6000, 0.0122, 4.437)
    df = cloud_free.spectrum(opa)
    wno_nc, alb_nc = jdi.mean_regrid(df['wavenumber'] , df['albedo'],R=150)
    df = cloudy.spectrum(opa)
    wno_c, alb_c = jdi.mean_regrid(df['wavenumber'] , df['albedo'],R=150)
    fig_no_cloud.line(1e4/wno_nc, alb_nc, legend_label=approx, color=c, line_width=3)
    fig_cloudy.line(1e4/wno_c, alb_c,  color=c, line_width=3)

jpi.plot_format(fig_cloudy)
jpi.plot_format(fig_no_cloud)
show(column(fig_no_cloud,fig_cloudy ))

## The Effect of Stellar Spectrum

With the updated Raman scattering approximation, you will notice imprints of the stellar spectrum in the planet reflected light spectrum. Take a look below.

In [13]:
fig_cloudy = figure(x_range=[0.3,1],y_range=[0,1.2], width=500, height=300
                    ,x_axis_label='Wavelength [μm]', y_axis_label='Geometric Albedo',)
fig_no_cloud = figure(x_range=[0.3,1],y_range=[0,1.2], width=500, height=300
                      ,x_axis_label='Wavelength [μm]', y_axis_label='Geometric Albedo',)

#lets play around with different stellar spectra Teff
stellar_teff = [6000,4000,3500]

colors = Colorblind8[0:len(options)]

cloud_free.approx(raman = 'oklopcic')
cloudy.approx(raman = 'oklopcic')

for approx, c in zip(stellar_teff, colors):
    cloud_free.star(opa, approx, 0.0122, 4.437) 
    cloudy.star(opa, approx ,0.0122, 4.437) 
    df = cloud_free.spectrum(opa)
    wno_nc, alb_nc = jdi.mean_regrid(df['wavenumber'] , df['albedo'],R=150)
    df = cloudy.spectrum(opa)
    wno_c, alb_c = jdi.mean_regrid(df['wavenumber'] , df['albedo'],R=150)
    fig_no_cloud.line(1e4/wno_nc, alb_nc, legend_label=str(approx), color=c, line_width=3)
    fig_cloudy.line(1e4/wno_c, alb_c,  color=c, line_width=3)

cloud_free.approx(raman = 'pollack')
cloudy.approx(raman = 'pollack')

df = cloud_free.spectrum(opa)
wno_nc, alb_nc = jdi.mean_regrid(df['wavenumber'] , df['albedo'],R=150)
df = cloudy.spectrum(opa)
wno_c, alb_c =jdi.mean_regrid( df['wavenumber'] , df['albedo'],R=150)
fig_no_cloud.line(1e4/wno_nc, alb_nc, legend_label='Pollack', color='black', line_width=2, line_dash='dashed')
fig_cloudy.line(1e4/wno_c, alb_c,  color='black', line_width=2, line_dash='dashed')


jpi.plot_format(fig_cloudy)
jpi.plot_format(fig_no_cloud)
show(column(fig_no_cloud,fig_cloudy ))